In [1]:
# Install the package if not already available
try:
    import owid.grapher
except ImportError:
    %pip install owid-grapher-py

try:
    %load_ext autoreload
    %autoreload 2
except:
    pass  # autoreload not available (e.g., in Google Colab with Python 3.12+)

import pandas as pd
from owid.grapher import Chart

# OWID Grapher Python Demo

This notebook demonstrates the `owid-grapher-py` package for creating interactive [Our World in Data](https://ourworldindata.org) charts in Jupyter notebooks.

The API is inspired by Altair: wrap your DataFrame in a `Chart`, specify marks (`.mark_line()`, `.mark_bar()`, `.mark_scatter()`), and encode your data dimensions.

## 1. Basic Examples with Sample Data

Let's start with some simple examples using sample population data.

In [2]:
# Sample population data (in millions)
df = pd.DataFrame({
    'year': [2000, 2005, 2010, 2015, 2020] * 3,
    'country': ['Australia'] * 5 + ['New Zealand'] * 5 + ['Japan'] * 5,
    'population': [
        19.2, 20.4, 22.0, 23.8, 25.7,  # Australia
        3.9, 4.1, 4.4, 4.6, 5.1,       # New Zealand
        126.8, 127.8, 128.1, 127.1, 125.8  # Japan
    ]
})
df

year      country  population
0   2000    Australia        19.2
1   2005    Australia        20.4
2   2010    Australia        22.0
3   2015    Australia        23.8
4   2020    Australia        25.7
5   2000  New Zealand         3.9
6   2005  New Zealand         4.1
7   2010  New Zealand         4.4
8   2015  New Zealand         4.6
9   2020  New Zealand         5.1
10  2000        Japan       126.8
11  2005        Japan       127.8
12  2010        Japan       128.1
13  2015        Japan       127.1
14  2020        Japan       125.8

### Line Chart

The most common chart type for time series data.

In [14]:
Chart(df).mark_line().encode(
    x='year',
    y='population',
    entity='country'
).label(title='Population Over Time').variable(
    column='population',
    unit='millions',
    short_unit='m'
)

### Bar Chart

Good for comparing values at a single point in time.

In [4]:
# Latest year only for bar chart
df_2020 = df[df['year'] == 2020]

Chart(df_2020).mark_bar().encode(
    x='population',
    y='country'
).label(title='Population in 2020')

### Simple Scatter Plot

For showing relationships between two variables.

In [5]:
# Create simple GDP vs Life Expectancy data
df_scatter = pd.DataFrame({
    'gdp_per_capita': [5000, 15000, 25000, 35000, 45000, 8000, 18000, 28000, 38000,
                       6000, 12000, 20000, 30000, 40000, 7000, 16000, 24000, 32000],
    'life_expectancy': [65, 72, 78, 81, 83, 68, 74, 79, 82,
                       66, 70, 76, 80, 81, 67, 73, 77, 80],
    'country': ['Australia'] * 6 + ['New Zealand'] * 6 + ['Japan'] * 6,
    'year': [2020, 2021, 2022, 2023, 2024, 2020] * 3
})

Chart(df_scatter).mark_scatter().encode(
    x='gdp_per_capita',
    y='life_expectancy',
    entity='country'
).label(
    title='GDP per Capita vs Life Expectancy',
    subtitle='Each point represents a different time period',
    source_desc='Sample data'
)

## 2. Real-World Example: Life Expectancy vs GDP

Let's recreate an actual OWID chart using real data. This example shows the full power of the library with color encoding, size encoding, log scales, and interactive controls.

Original chart: https://ourworldindata.org/grapher/life-expectancy-un-vs-gdp-per-capita-wb

In [6]:
# Fetch data directly from OWID's CSV endpoint
df_real = pd.read_csv(
    "https://ourworldindata.org/grapher/life-expectancy-un-vs-gdp-per-capita-wb.csv?useColumnShortNames=true"
)

# Rename columns for consistency
df_real = df_real.rename(columns={'Entity': 'entity', 'Year': 'year'})

# CSV has region info only for last year, propagate it to all rows
df_real['owid_region'] = df_real.entity.map(
    df_real.dropna(subset='owid_region').set_index('entity')['owid_region']
).values

print(f"Loaded {len(df_real):,} rows of data")
df_real.head()

Loaded 59,664 rows of data


entity Code  year  life_expectancy__sex_all__age_0__variant_estimates  \
0  Afghanistan  AFG  1950                                             28.156    
1  Afghanistan  AFG  1951                                             28.584    
2  Afghanistan  AFG  1952                                             29.014    
3  Afghanistan  AFG  1953                                             29.452    
4  Afghanistan  AFG  1954                                             29.698    

   ny_gdp_pcap_pp_kd  population_historical owid_region  
0                NaN              7776180.0        Asia  
1                NaN              7879343.0        Asia  
2                NaN              7987784.0        Asia  
3                NaN              8096703.0        Asia  
4                NaN              8207954.0        Asia

In [7]:
# Create a fully-featured scatter plot
Chart(df_real).mark_scatter().encode(
    x="ny_gdp_pcap_pp_kd",
    y="life_expectancy__sex_all__age_0__variant_estimates",
    entity="entity",
    color="owid_region",      # Color by region
    size="population_historical"  # Size by population
).label(
    title="Life Expectancy vs GDP per Capita",
    source_desc="UN World Population Prospects; World Bank"
).xaxis(
    label="GDP per Capita (constant 2015 US$)",
    unit="$",
    scale="log",              # Logarithmic scale
    scale_control=True        # Allow user to toggle log/linear
).yaxis(
    label="Life Expectancy (years)",
    unit="years"
).filter(
    matching_entities_only=True  # Only show entities with both variables
).interact(
    entity_control=True       # Enable country picker
)

### CO2 Emissions with Map View

Another real-world example showing global CO2 emissions over time with interactive map visualization.

Original chart: https://ourworldindata.org/grapher/annual-co2-emissions-per-country

In [8]:
# Fetch CO2 emissions data from OWID
df_co2 = pd.read_csv(
    "https://ourworldindata.org/grapher/annual-co2-emissions-per-country.csv?useColumnShortNames=true"
)

# Rename columns for consistency
df_co2 = df_co2.rename(columns={'Entity': 'entity', 'Year': 'year'})

print(f"Loaded {len(df_co2):,} rows of CO2 data")
print(f"Time range: {df_co2['year'].min()} to {df_co2['year'].max()}")
df_co2.head()

Loaded 29,384 rows of CO2 data
Time range: 1750 to 2024


entity Code  year  emissions_total
0  Afghanistan  AFG  1949          14656.0
1  Afghanistan  AFG  1950          84272.0
2  Afghanistan  AFG  1951          91600.0
3  Afghanistan  AFG  1952          91600.0
4  Afghanistan  AFG  1953         106256.0

In [9]:
# Create line chart with map view for CO2 emissions
Chart(df_co2).mark_line().mark_map().encode(
    x="year",
    y="emissions_total",
    entity="entity"
).label(
    title="Annual CO₂ emissions",
    subtitle="Carbon dioxide emissions from fossil fuels and industry",
    source_desc="Global Carbon Project"
).yaxis(
    label="CO₂ emissions",
    unit="tonnes"
).select(
    # Pre-select major emitting countries
    entities=["United States", "China", "India", "United Kingdom", "Germany", "France", "Brazil"]
).interact(
    allow_relative=True,    # Allow viewing as relative change
    entity_control=True     # Enable country picker
)

### Map Configuration

The `map()` method allows you to customize the map tab appearance with color schemes and binning strategies.

In [10]:
# Configure map with custom color scheme and binning
Chart(df_co2).mark_line().mark_map(
    color_scheme='OrRd',          # Orange-Red color scheme
    binning_strategy='auto'
).show("map").encode(
    x="year",
    y="emissions_total",
    entity="entity"
).label(
    title="Annual CO₂ emissions",
    subtitle="With custom map styling",
    source_desc="Global Carbon Project"
).interact(
    entity_control=True
)

## 3. Interactive Features

OWID charts support various interactive controls that users can toggle.

### Entity Picker

Let users select which countries/entities to display.

In [11]:
Chart(df).mark_line().encode(
    x='year',
    y='population',
    entity='country'
).label(
    title='Population Trends',
    subtitle='Select countries to compare'
).interact(
    entity_control=True
)

### Map View

For geographic data, enable the map tab.

In [12]:
Chart(df).mark_line().mark_map().show("map").encode(
    x='year',
    y='population',
    entity='country'
).label(title='Population by Country')

## 4. Export Configuration

For advanced users: view the underlying JSON configuration that gets sent to the OWID Grapher.

In [13]:
chart = Chart(df).mark_line().encode(
    x='year',
    y='population',
    entity='country'
).label(title='Population Over Time')

# Export the configuration
chart.export()

{'csv_data': 'year,entityName,population\n2000,Australia,19.2\n2005,Australia,20.4\n2010,Australia,22.0\n2015,Australia,23.8\n2020,Australia,25.7\n2000,New Zealand,3.9\n2005,New Zealand,4.1\n2010,New Zealand,4.4\n2015,New Zealand,4.6\n2020,New Zealand,5.1\n2000,Japan,126.8\n2005,Japan,127.8\n2010,Japan,128.1\n2015,Japan,127.1\n2020,Japan,125.8\n',
 'column_defs': [{'slug': 'population', 'type': 'Numeric'}],
 'grapher_config': {'chartTypes': ['LineChart'],
  'tab': 'line',
  'title': 'Population Over Time',
  'selectedEntityNames': ['Australia', 'New Zealand', 'Japan'],
  'ySlugs': 'population',
  'xAxis': {},
  'yAxis': {},
  'hideLogo': True}}

## Next Steps

Check out the [README](https://github.com/owid/owid-grapher-py) for more examples and documentation:

- Stacked bar charts
- Custom axis configuration
- Data selection and filtering
- Relative mode transformations

Explore more OWID charts at [ourworldindata.org](https://ourworldindata.org)